In [1]:
%matplotlib inline

from time import sleep
import random

import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
_MIN, _MAX = 0, 10
_TARGET = 5

class MysteryGame:
    
    def __init__(self):
        state = self.reset()

        
    def reset(self):
        d = None
        while True:
            d = random.randint(_MIN, _MAX)
            if d != _TARGET:
                break
            
        self.state = d
        self.turn = 0
        return [self.state]
    
    def step(self, action):
        done = False
#         reward = _TARGET - abs(_TARGET - self.state)
#         reward = 0
        reward =  -abs(_TARGET - self.state)
        if action == 0:
            self.state += 1
        elif action == 1:
            self.state -= 1
        
        if self.state > _MAX or self.state < _MIN or self.turn > _MAX + 10:
            done = True
            reward = -100
        elif self.state == _TARGET:
            done = True
            reward = 100
        self.turn += 1
        return [self.state], reward, done, {}
    
env = MysteryGame()
        

In [3]:
# _input = tf.div(tf.placeholder(shape=[1, 1], dtype=tf.float32), _MAX)
_input = tf.placeholder(shape=[1, 1], dtype=tf.float32)
weight = tf.Variable(tf.random_uniform([1, 2], -0.1, 0.1))
bias = tf.Variable(tf.random_uniform([1, 2], -0.1, 0.1))
out = tf.add(tf.multiply(_input, weight), bias)
select_action = tf.argmax(out, 1)

expect_weight = tf.placeholder(shape=[1, 2], dtype=tf.float32)
error = tf.reduce_sum(tf.square(expect_weight - weight))
trainer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(error)

In [4]:
# _input = tf.placeholder(shape=[1, 1], dtype=tf.float32)
# i0 = tf.tanh(_input)
# w1 = tf.Variable(tf.random_uniform([1, 2], -0.1, 0.1))
# # b1 = tf.Variable(tf.random_uniform([1, 2], -0.1, 0.1))
# # z1 = tf.add(tf.matmul(_input, w1), b1)
# # Q = tf.tanh(w1)

# # w2 = tf.Variable(tf.random_uniform([2, 2], -0.01, 0.01))
# # b2 = tf.Variable(tf.random_uniform([1, 2], -0.01, 0.01))
# # z2 = tf.add(tf.matmul(h1, w2), b2)
# # Q = tf.tanh(z2)
            
# Qa = tf.argmax(Q, 1)
# Qe = tf.placeholder(shape=[1, 2], dtype=tf.float32)
# error = tf.reduce_sum(tf.square(Qe - Q))
# trainer = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(error)

In [5]:
nactions =  2
nparams = 4
learning_rate = 0.05
gamma = .3

beta = 0.9
nepisode = 50

nbatch = 100
batch_legth = 512
batch_iter = 100


In [6]:
init = tf.global_variables_initializer()
steps = [0]
with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        state = env.reset()
        done = False
        step = 0
        r = 0
        while done == False:
            step += 1
            res_action, res_weight = sess.run([select_action, weight], feed_dict={_input:[state]})
#             print(state, res_weight)
            if  np.random.rand(1) < beta:
                new_action = random.choice([0, 1])
                res_action[0] = new_action
            
            # do action 
            new_state, reward, done, _ = env.step(res_action[0])
            next_Q = sess.run(weight, feed_dict={_input:[new_state]})
            y = reward
            if not done:
                y = reward + gamma*np.max(next_Q)
                
                
            qn = res_weight.copy()
            if state[0] > _TARGET:
                qn[0, 1] = y
                print(state, 1)
            else:
                qn[0, 0] = y
                print(state, 0)
            print(qn)
            _, w, ew = sess.run([trainer, weight, expect_weight], feed_dict={_input:[state], expect_weight:qn})
#             print('\t', state, qn, w, ew)
#             print()
            state = new_state

        # play
        print()
        done = False
        state = env.reset()
        while not done:
            qa, q = sess.run([select_action, weight], feed_dict={_input:[state]})
            state, reward, done, _ = env.step(qa[0])
#             print(state, q, done, reward)
    
    for i in range(_MIN, _MAX+1):
        res_action, res_weight, outp = sess.run([select_action, weight, out], feed_dict={_input:[[i]]})
        print("{}: {}, {}".format(i, res_action, outp))
        
# plt.plot(steps)

[10] 1
[[  6.49442151e-02  -1.00000000e+02]]

[8] 1
[[ 0.06494422 -2.98051667]]
[7] 1
[[ 0.06494422 -1.98051679]]
[8] 1
[[ 0.06494422 -2.98051667]]
[9] 1
[[ 0.06494422 -3.98051667]]
[8] 1
[[ 0.06494422 -2.98051667]]
[9] 1
[[ 0.06494422 -3.98051667]]
[8] 1
[[ 0.06494422 -2.98051667]]
[9] 1
[[ 0.06494422 -3.98051667]]
[10] 1
[[ 0.06494422 -4.98051691]]
[9] 1
[[ 0.06494422 -3.98051667]]
[8] 1
[[ 0.06494422 -2.98051667]]
[9] 1
[[ 0.06494422 -3.98051667]]
[10] 1
[[  6.49442151e-02  -1.00000000e+02]]

[1] 0
[[-3.98051667 -0.06879871]]
[0] 0
[[-4.9819746  -0.07338981]]
[1] 0
[[-3.98394632 -0.07755051]]
[0] 0
[[ -1.00000000e+02  -8.13254863e-02]]

[10] 1
[[  4.03595679e-02  -1.00000000e+02]]

[8] 1
[[ 0.03537467 -2.98938751]]
[7] 1
[[ 0.03084136 -1.99074757]]
[8] 1
[[ 0.02671666 -2.99198508]]
[7] 1
[[ 0.02296225 -1.99311137]]
[8] 1
[[ 0.01954381 -2.99413681]]
[9] 1
[[ 0.01643053 -3.99507093]]
[8] 1
[[ 0.01359467 -2.99592161]]
[9] 1
[[ 0.01101117 -3.99669671]]
[8] 1
[[ 0.0086574  -2.99740267]]


[1] 0
[[-3.9506433  -0.77802783]]
[0] 0
[[-4.95057011 -0.77690911]]
[1] 0
[[-3.95056486 -0.77590036]]
[0] 0
[[-100.          -0.7749908]]

[10] 1
[[   0.16061261 -100.        ]]

[10] 1
[[   0.15698656 -100.        ]]

[4] 0
[[ 100.           -0.78240234]]

[3] 0
[[-1.95360708 -0.78745723]]
[2] 0
[[-2.95338082 -0.79201514]]
[1] 0
[[-3.95321226 -0.79612488]]
[0] 0
[[-100.           -0.79983056]]

[8] 1
[[ 0.15288264 -2.95413518]]
[9] 1
[[ 0.14979318 -3.95506215]]
[10] 1
[[   0.14700754 -100.        ]]

[1] 0
[[-3.95665121 -0.814964  ]]
[0] 0
[[-100.           -0.82021201]]

[4] 0
[[ 100.           -0.82494384]]

[0] 0
[[-4.95957327 -0.82921028]]
[1] 0
[[-3.96004009 -0.83305705]]
[2] 0
[[-2.96050525 -0.83652544]]
[1] 0
[[-3.960958   -0.83965266]]
[2] 0
[[-2.96141052 -0.8424722 ]]
[3] 0
[[-1.96185195 -0.84501439]]
[2] 0
[[-2.96227288 -0.84730649]]
[1] 0
[[-3.96268582 -0.84937304]]
[2] 0
[[-2.96310282 -0.85123628]]
[1] 0
[[-3.96351266 -0.85291618]]
[0] 0
[[-4.96392679 -0.85443079]]
[1] 0
[

[1] 0
[[-3.98683596 -0.96760601]]
[0] 0
[[-100.           -0.96954072]]

[2] 0
[[-2.98895311 -0.97128385]]
[1] 0
[[-3.99039268 -0.97285438]]
[2] 0
[[-2.9917326  -0.97426939]]
[1] 0
[[-3.99297237 -0.97554427]]
[0] 0
[[-4.99413252 -0.97669291]]
[1] 0
[[-3.99523139 -0.97772783]]
[2] 0
[[-2.9962647  -0.97866029]]
[3] 0
[[-1.99722803 -0.97950035]]
[4] 0
[[-0.99811757 -0.98025727]]
[3] 0
[[-1.99892986 -0.98093921]]
[4] 0
[[ 100.           -0.98155361]]

[3] 0
[[-1.99928665 -0.98210716]]
[2] 0
[[-2.99895072 -0.98260593]]
[3] 0
[[-1.99868011 -0.98305529]]
[2] 0
[[-2.99845767 -0.98346013]]
[3] 0
[[-1.99828947 -0.98382491]]
[2] 0
[[-2.99815941 -0.98415357]]
[3] 0
[[-1.99807453 -0.98444968]]
[4] 0
[[ 100.           -0.98471648]]

[8] 1
[[ 0.01030072 -2.99690986]]
[9] 1
[[ 0.01363337 -3.99590993]]
[8] 1
[[ 0.01663595 -2.99500918]]
[9] 1
[[ 0.01934114 -3.99419761]]
[10] 1
[[  2.17784047e-02  -1.00000000e+02]]

[9] 1
[[ 0.02397427 -3.99280763]]
[10] 1
[[ 0.02595264 -4.9922142 ]]
[9] 1
[[ 0.02773506 